In [71]:
sc

In [70]:
data = [("a", 4), ("b", 3),("a", 1), ("b", 1), ("a", 9)]
rdd = sc.parallelize(data,3)

In [74]:
rdd.collect()
rdd.combineByKey(str,  lambda x : x+str, lambda x,y:x+y).collect()

[('b', '31'), ('a', '419')]

In [ ]:
data = [
        ("A", 2.), ("Z", 4.), ("A", 9.), 
        ("B", 10.), ("B", 20.), 
        ("Z", 3.), ("Z", 5.), ("Z", 8.), ("A", 12.) 
       ]

In [41]:
rdd = sc.parallelize(data,3)

In [42]:
def printPartition(iterator):
    nstr = ''
    for x in iterator:
        nstr = nstr +'['+ str(x[0]) + ',' + str(x[1])+']'
    yield nstr
        

rdd.mapPartitions(printPartition).collect()

['[A,2.0][Z,4.0][A,9.0]', '[B,10.0][B,20.0][Z,3.0]', '[Z,5.0][Z,8.0][A,12.0]']

In [48]:
combineRdd = rdd.combineByKey(lambda value : (value,1), 
                lambda accu,x : (x + accu[0] , 1 + accu[1]),
                 lambda x,y : (x[0] + y[0] , x[1] + y[1]))

In [49]:
combineRdd.collect()

[('A', (23.0, 3)), ('Z', (20.0, 4)), ('B', (30.0, 2))]

In [50]:
#Compute mean
averageByKey = combineRdd.map(lambda (key, (totalSum, count)): (key, totalSum / count))

In [51]:
averageByKey.collectAsMap()

{'A': 7.666666666666667, 'B': 15.0, 'Z': 5.0}

In [52]:
averageByKey.collect()

[('A', 7.666666666666667), ('Z', 5.0), ('B', 15.0)]

In [53]:
#Standard Deviation and MEAN both
sumCount = rdd.combineByKey(lambda value: (value, value*value, 1),
                             lambda x, value: (x[0] + value, x[1] + value*value, x[2] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2])
                            )

In [61]:
sumCount.collect()
sumCount.countByKey()

defaultdict(int, {'A': 1, 'B': 1, 'Z': 1})

In [59]:
import math
def  stdDev( sumX, sumSquared, n ):     
    mean = sumX / n
    stdDeviation = math.sqrt ((sumSquared - n*mean*mean) /n)
    return (mean, stdDeviation)
meanAndStdDev = sumCount.mapValues(lambda x : stdDev(x[0], x[1], x[2]))
meanAndStdDev.collect()

[(7.666666666666667, 4.189935029992178),
 (5.0, 1.8708286933869707),
 (15.0, 5.0)]